In [ ]:
import numpy as np
import os, sys
import matplotlib.pyplot as plt

import pyrr
import itertools

# basedir = os.path.dirname(os.getcwd())
basedir = os.path.abspath(os.path.join(os.getcwd() ,"../"))
_py = os.path.join(basedir, 'py')
_data = os.path.join(basedir, 'data')

sys.path.insert(1, _py)
import loads
import lia
import ray as rayt
import lad
import figures

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

%matplotlib qt

In [ ]:
mockname = 'test_kiwi_2'

In [ ]:
def segtree(df, leaves, show=False):

    trees = {}

    if show:
        plt.figure(figsize=(14, 8))

    # centres
    x, y = [0], [0]
    num = 0
    dx, dy = 2, 2
    # dx, dy = 5, 5

    for i in x:
        for j in y:
            
            keep = np.ones(len(df['x']), dtype=bool)
            keep &= (df['x'] < i+dx) & (df['x'] > i-dx)
            keep &= (df['y'] < j+dy) & (df['y'] > j-dy)

            trees['tree_%s' %(str(num))] = keep
            
            if show:
                plt.scatter(df['x'][leaves & keep], df['y'][leaves & keep], s=0.5, label=num)
                        
            num += 1

    if show:
        plt.legend()
    
    return trees

In [ ]:
# load data into a pandas data frame
df = loads.npy2pandas(mockname)
N = len(df)

In [ ]:

def voxel_subsampling(voxel_size, POINTS):

    nb_vox = np.ceil((np.max(POINTS, axis=0) - np.min(POINTS, axis=0))/voxel_size)
    ni, nj, nk = nb_vox
    print('Number of voxels: i:%d, j:%d, k:%d --> Total: %d' %(ni, nj, nk, np.product(nb_vox)))

    non_empty_voxel_keys, inverse, nb_pts_per_voxel = np.unique(((POINTS - np.min(POINTS, axis=0)) // voxel_size).astype(int), axis=0, return_inverse=True, return_counts=True)
    idx_pts_vox_sorted = np.argsort(inverse)
    print('Number of non-empty voxels: %d' %(len(non_empty_voxel_keys)))

    voxel_grid={}
    voxel_grid_ptsidx = {}
    grid_barycenter,grid_candidate_center = [], []
    last_seen=0

    for idx, vox in enumerate(non_empty_voxel_keys):

        idxs_per_vox = idx_pts_vox_sorted[last_seen:last_seen+nb_pts_per_voxel[idx]]
        voxel_grid[tuple(vox)] = POINTS[idxs_per_vox]
        voxel_grid_ptsidx[tuple(vox)] = idxs_per_vox

        # grid_barycenter.append(np.mean(voxel_grid[tuple(vox)],axis=0))

        idx_grid_candidate_center = np.linalg.norm(voxel_grid[tuple(vox)] - np.mean(voxel_grid[tuple(vox)],axis=0),axis=1).argmin()
        grid_candidate_center.append(voxel_grid_ptsidx[tuple(vox)][idx_grid_candidate_center])

        last_seen+=nb_pts_per_voxel[idx]

    print('Downsampling percentage: %.1f %%' %(100 * len(grid_candidate_center) / len(POINTS)))

    return list(grid_candidate_center)

In [ ]:
inds = voxel_subsampling(0.08, df[['x', 'y', 'z']].to_numpy())

df = df.iloc[inds]
POINTS = df[['x', 'y', 'z']].to_numpy()
SENSORS = df[['sx', 'sy', 'sz']].to_numpy()

In [ ]:
# extract leaves. Boolean array output
leaves = loads.extract_leaves(df, show=True)
# extract trees. Dictionary with boolean arrays output
trees = segtree(df, leaves, show=True)

In [ ]:
inPR = (leaves) & (trees['tree_0'])
minBB, maxBB = np.min(POINTS[inPR.values], axis=0), np.max(POINTS[inPR.values], axis=0)
boxPR = pyrr.aabb.create_from_bounds(minBB, maxBB)

lines = np.stack((POINTS, SENSORS), axis=1)
f = lambda line: pyrr.geometric_tests.ray_i

In [ ]:
POINTS, SENSORS = POINTS[res], SENSORS[res]

leaves = leaves[res]

for key, val in trees.items():
    trees[key] = val[res]

In [ ]:
# save indexes of voxel-based downsample

idxs = np.array(inds)[res]
voxel_size = 0.08

resdir = os.path.join(_data, mockname, 'lad_%s' %(str(voxel_size)))
outdir = os.path.join(resdir, 'inds.npy')
np.save(outdir, idxs)

In [ ]:
# get numpy array of AABBs for each voxel

voxel_size = 0.08
keep = (trees['tree_0']) & (leaves)

nb_vox = np.ceil((np.max(POINTS[keep.values], axis=0) - np.min(POINTS[keep.values], axis=0))/voxel_size)
ni, nj, nk = nb_vox
print('Number of voxels: i:%d, j:%d, k:%d --> Total: %d' %(ni, nj, nk, np.product(nb_vox)))

minBB = np.min(POINTS[keep.values], axis=0) + (np.array(np.meshgrid(range(0,int(ni)), range(0, int(nj)), range(0, int(nk)))).T.reshape(-1,3) * voxel_size)
maxBB = minBB + voxel_size

AABBs = np.stack((minBB, maxBB), axis=1)
rays = np.stack((POINTS, SENSORS), axis=1)

In [ ]:
f = lambda comb: pyrr.geometric_tests.ray_intersect_aabb(pyrr.ray.create_from_line(comb[0]), comb[1]) is not None
seq = list(itertools.product(rays[:20], AABBs))
boxes = map(f, seq)

In [ ]:
# get AABBs centres

AABB_centres = np.mean(AABBs, axis=1)

# get points within cylinder centred at ray path

def points_in_cylinder(ray, r, q):

    ra, rb = ray
    e = rb - ra
    m = np.cross(ra, rb)
    const = r * np.linalg.norm(e)

    f = lambda qi: (np.dot(qi - ra, e) >= 0) & (np.dot(qi - rb, e) <= 0) & (np.linalg.norm(np.cross(e, qi - ra)) <= const)

    return np.where([f(i) for i in q])

res = {}
for num, ray in enumerate(rays[:50]):
    res[num] = points_in_cylinder(ray, 0.1, AABB_centres)

In [ ]:
res = {}

for num, ray in enumerate(rays[:50]):
    # res[num] = [AABB for AABB in AABBs[points_in_cylinder(ray, 0.08, AABB_centres)[0]] if pyrr.geometric_tests.ray_intersect_aabb(pyrr.ray.create_from_line(ray), AABB) is not None]
    idxs = [AABB for AABB in AABBs[points_in_cylinder(ray, 0.08, AABB_centres)[0]] if pyrr.geometric_tests.ray_intersect_aabb(pyrr.ray.create_from_line(ray), AABB) is not None]
    print(ray, idxs)

In [ ]:

def retrieve_AABB(rays, AABBs):

    for ray, AABB in list(itertools.product(rays[:2], AABBs)):

        if pyrr.geometric_tests.ray_intersect_aabb(pyrr.ray.create_from_line(ray), AABB) is not None:
            
            yield AABB, ray

# boxes = map(retrieve_AABB, AABBs)
boxes = retrieve_AABB(rays, AABBs)
# for AABB in AABBs:

# boxes = [next(retrieve_AABB(AABB)) for AABB in AABBs]

In [ ]:
res = []
while True:
    try:
        res.append(next(boxes))
    except StopIteration:
        break